Connected to ShinyHunterPuppetMaster (Python 3.11.0)

In [1]:
import re
import win32gui, win32ui , win32con, win32api
import numpy as np
from dataclasses import dataclass, field ,fields

@dataclass
class Window:
    """
    Dataclass for window.
    """    
    hwnd:   int
    title:  str = field(init=False)
    left:   int = field(init=False)
    top:    int = field(init=False)
    right:  int = field(init=False) 
    bot:    int = field(init=False)
    w:      int = field(init=False)
    h:      int = field(init=False)
    
    def __post_init__(self):
        self.title = win32gui.GetWindowText(self.hwnd)
        self.left, self.top, self.right, self.bot = win32gui.GetWindowRect(self.hwnd)
        w = self.right - self.left
        h = self.bot - self.top

In [2]:
def list_visable_windows():
    """
    Print all visable windows.
    """
    win_list = []
    def winEnumHandler(hwnd, ctx):
        if win32gui.IsWindowVisible(hwnd):
            name_list.append(Window(hwnd))
            # print(hex(hwnd), win32gui.GetWindowText(hwnd))
    win32gui.EnumWindows(winEnumHandler, None)
    #remove empty strings
    name_list = [x for x in win_list if x.name != '']
    return name_list    

def grab_screen():
    w = 1920 # set this
    h = 1080 # set this
    bmpfilenamename = "out.bmp" #set this

    hwnd= None #desktop
    wDC = win32gui.GetWindowDC(hwnd) # type: ignore
    dcObj=win32ui.CreateDCFromHandle(wDC)
    cDC=dcObj.CreateCompatibleDC()
    dataBitMap = win32ui.CreateBitmap()
    dataBitMap.CreateCompatibleBitmap(dcObj, w, h)
    cDC.SelectObject(dataBitMap)
    cDC.BitBlt((0,0),(w, h) , dcObj, (0,0), win32con.SRCCOPY)
    dataBitMap.SaveBitmapFile(cDC, bmpfilenamename)

    # Free Resources
    dcObj.DeleteDC()
    cDC.DeleteDC()
    win32gui.ReleaseDC(hwnd, wDC) # type: ignore
    win32gui.DeleteObject(dataBitMap.GetHandle())

def grab_window_bitmap(title = None):  # sourcery skip: raise-specific-error
    """
    Grab a screenshot of the specified window.

    Args:
        title (_type_, optional): _description_. Defaults to None.
    """
    
    w = 1920 # set this
    h = 1080 # set this
    # get the window handle
    hwnd = win32gui.FindWindow(None, title)
    if not hwnd:
        raise Exception(f"Window {title} not found.")
    '''
    cDC.SelectObject(dataBitMap)
    cDC.BitBlt((0, 0), (self.w, self.h), dcObj, (self.cropped_x, self.cropped_y), win32con.SRCCOPY)
    '''
    # get the window size
    react = win32gui.GetWindowRect(hwnd)
    # left, top, right, bot = win32gui.GetWindowRect(hwnd)
    # w = right - left
    # h = bot - top
    
    wDC = win32gui.GetWindowDC(hwnd) # type: ignore
    dcObj=win32ui.CreateDCFromHandle(wDC)
    cDC=dcObj.CreateCompatibleDC()
    dataBitMap = win32ui.CreateBitmap()
    dataBitMap.CreateCompatibleBitmap(dcObj, w, h)
    cDC.SelectObject(dataBitMap)
    cDC.BitBlt((0,0),(w, h) , dcObj, (0,0), win32con.SRCCOPY)
    signedIntsArray = dataBitMap.GetBitmapBits(True)
    # Free Resources
    dcObj.DeleteDC()
    cDC.DeleteDC()
    win32gui.ReleaseDC(hwnd, wDC) # type: ignore
    win32gui.DeleteObject(dataBitMap.GetHandle())
    img = np.fromstring(signedIntsArray, dtype='uint8') # type: ignore
    
    img.shape = (h,w,4)
    # cut off the alpha channel
    # img = img[:,:,:3]
    return img
    
def grab_save_window_screenshot(hwnd=None):
    
    """
    Grab a screenshot of the current window.
    """
    w = 1920 # set this
    h = 1080 # set this
    # get the window handle
    # hwnd = win32gui.FindWindow(None, title)
    # if not hwnd:
    #     raise Exception(f"Window {title} not found.")
    '''
    cDC.SelectObject(dataBitMap)
    cDC.BitBlt((0, 0), (self.w, self.h), dcObj, (self.cropped_x, self.cropped_y), win32con.SRCCOPY)
    '''
    # get the window size
    react = win32gui.GetWindowRect(hwnd) # type: ignore
    left, top, right, bot = win32gui.GetWindowRect(hwnd) # type: ignore
    w = right - left
    h = bot - top
    
    wDC = win32gui.GetWindowDC(hwnd) # type: ignore
    dcObj=win32ui.CreateDCFromHandle(wDC)
    cDC=dcObj.CreateCompatibleDC()
    dataBitMap = win32ui.CreateBitmap()
    dataBitMap.CreateCompatibleBitmap(dcObj, w, h)
    cDC.SelectObject(dataBitMap)
    cDC.BitBlt((left,top),(w, h) , dcObj, (0,0), win32con.SRCCOPY)
    dataBitMap.SaveBitmapFile(cDC, f'img_bank/pic_{win32gui.GetWindowText(hwnd)}_{hwnd}.bmp') # type: ignore
    # Free Resources
    dcObj.DeleteDC()
    cDC.DeleteDC()
    win32gui.ReleaseDC(hwnd, wDC) # type: ignore
    win32gui.DeleteObject(dataBitMap.GetHandle())
    # img = np.fromstring(signedIntsArray, dtype='uint8') # type: ignore
    # img.shape = (h,w,4)
    # cut off the alpha channel
    # img = img[:,:,:3]
    # return img
    # ------------------------------------
    # hwin = None
    # if title == None:
    #     hwin = win32gui.FindWindow(None, title)
    
    # left, top, right, bot = win32gui.GetWindowRect(hwin)
    # w = 1920
    # h = 1080
    
    # # save the screen
    # hwndDC = win32gui.GetWindowDC(hwin)
    # mfcDC  = win32ui.CreateDCFromHandle(hwndDC)
    # saveDC = mfcDC.CreateCompatibleDC()
    # saveBitMap = win32ui.CreateBitmap()
    # saveBitMap.CreateCompatibleBitmap(mfcDC, w, h)
    # saveDC.SelectObject(saveBitMap)
    # saveDC.BitBlt((0, 0), (w, h),  mfcDC,  (0, 0),  win32con.SRCCOPY)
    
    # #save to file
    # saveBitMap.SaveBitmapFile(saveDC, f'img_bank/pic_{title}.bmp')
    
    # mfcDC.DeleteDC()
    # saveDC.DeleteDC()
    # #realease resources
    # win32gui.ReleaseDC(hwin, hwndDC)
    # win32gui.DeleteObject(saveBitMap.GetHandle())

In [3]:
ls = list_visable_windows()

NameError: cannot access free variable 'name_list' where it is not associated with a value in enclosing scope

In [4]:
def list_visable_windows():
    """
    Print all visable windows.
    """
    win_list = []
    def winEnumHandler(hwnd, ctx):
        if win32gui.IsWindowVisible(hwnd):
            name_list.append(Window(hwnd))
            # print(hex(hwnd), win32gui.GetWindowText(hwnd))
    win32gui.EnumWindows(winEnumHandler, None)
    #remove empty strings
    win_list = [x for x in win_list if x.name != '']
    return win_list    

def grab_screen():
    w = 1920 # set this
    h = 1080 # set this
    bmpfilenamename = "out.bmp" #set this

    hwnd= None #desktop
    wDC = win32gui.GetWindowDC(hwnd) # type: ignore
    dcObj=win32ui.CreateDCFromHandle(wDC)
    cDC=dcObj.CreateCompatibleDC()
    dataBitMap = win32ui.CreateBitmap()
    dataBitMap.CreateCompatibleBitmap(dcObj, w, h)
    cDC.SelectObject(dataBitMap)
    cDC.BitBlt((0,0),(w, h) , dcObj, (0,0), win32con.SRCCOPY)
    dataBitMap.SaveBitmapFile(cDC, bmpfilenamename)

    # Free Resources
    dcObj.DeleteDC()
    cDC.DeleteDC()
    win32gui.ReleaseDC(hwnd, wDC) # type: ignore
    win32gui.DeleteObject(dataBitMap.GetHandle())

def grab_window_bitmap(title = None):  # sourcery skip: raise-specific-error
    """
    Grab a screenshot of the specified window.

    Args:
        title (_type_, optional): _description_. Defaults to None.
    """
    
    w = 1920 # set this
    h = 1080 # set this
    # get the window handle
    hwnd = win32gui.FindWindow(None, title)
    if not hwnd:
        raise Exception(f"Window {title} not found.")
    '''
    cDC.SelectObject(dataBitMap)
    cDC.BitBlt((0, 0), (self.w, self.h), dcObj, (self.cropped_x, self.cropped_y), win32con.SRCCOPY)
    '''
    # get the window size
    react = win32gui.GetWindowRect(hwnd)
    # left, top, right, bot = win32gui.GetWindowRect(hwnd)
    # w = right - left
    # h = bot - top
    
    wDC = win32gui.GetWindowDC(hwnd) # type: ignore
    dcObj=win32ui.CreateDCFromHandle(wDC)
    cDC=dcObj.CreateCompatibleDC()
    dataBitMap = win32ui.CreateBitmap()
    dataBitMap.CreateCompatibleBitmap(dcObj, w, h)
    cDC.SelectObject(dataBitMap)
    cDC.BitBlt((0,0),(w, h) , dcObj, (0,0), win32con.SRCCOPY)
    signedIntsArray = dataBitMap.GetBitmapBits(True)
    # Free Resources
    dcObj.DeleteDC()
    cDC.DeleteDC()
    win32gui.ReleaseDC(hwnd, wDC) # type: ignore
    win32gui.DeleteObject(dataBitMap.GetHandle())
    img = np.fromstring(signedIntsArray, dtype='uint8') # type: ignore
    
    img.shape = (h,w,4)
    # cut off the alpha channel
    # img = img[:,:,:3]
    return img
    
def grab_save_window_screenshot(hwnd=None):
    
    """
    Grab a screenshot of the current window.
    """
    w = 1920 # set this
    h = 1080 # set this
    # get the window handle
    # hwnd = win32gui.FindWindow(None, title)
    # if not hwnd:
    #     raise Exception(f"Window {title} not found.")
    '''
    cDC.SelectObject(dataBitMap)
    cDC.BitBlt((0, 0), (self.w, self.h), dcObj, (self.cropped_x, self.cropped_y), win32con.SRCCOPY)
    '''
    # get the window size
    react = win32gui.GetWindowRect(hwnd) # type: ignore
    left, top, right, bot = win32gui.GetWindowRect(hwnd) # type: ignore
    w = right - left
    h = bot - top
    
    wDC = win32gui.GetWindowDC(hwnd) # type: ignore
    dcObj=win32ui.CreateDCFromHandle(wDC)
    cDC=dcObj.CreateCompatibleDC()
    dataBitMap = win32ui.CreateBitmap()
    dataBitMap.CreateCompatibleBitmap(dcObj, w, h)
    cDC.SelectObject(dataBitMap)
    cDC.BitBlt((left,top),(w, h) , dcObj, (0,0), win32con.SRCCOPY)
    dataBitMap.SaveBitmapFile(cDC, f'img_bank/pic_{win32gui.GetWindowText(hwnd)}_{hwnd}.bmp') # type: ignore
    # Free Resources
    dcObj.DeleteDC()
    cDC.DeleteDC()
    win32gui.ReleaseDC(hwnd, wDC) # type: ignore
    win32gui.DeleteObject(dataBitMap.GetHandle())
    # img = np.fromstring(signedIntsArray, dtype='uint8') # type: ignore
    # img.shape = (h,w,4)
    # cut off the alpha channel
    # img = img[:,:,:3]
    # return img
    # ------------------------------------
    # hwin = None
    # if title == None:
    #     hwin = win32gui.FindWindow(None, title)
    
    # left, top, right, bot = win32gui.GetWindowRect(hwin)
    # w = 1920
    # h = 1080
    
    # # save the screen
    # hwndDC = win32gui.GetWindowDC(hwin)
    # mfcDC  = win32ui.CreateDCFromHandle(hwndDC)
    # saveDC = mfcDC.CreateCompatibleDC()
    # saveBitMap = win32ui.CreateBitmap()
    # saveBitMap.CreateCompatibleBitmap(mfcDC, w, h)
    # saveDC.SelectObject(saveBitMap)
    # saveDC.BitBlt((0, 0), (w, h),  mfcDC,  (0, 0),  win32con.SRCCOPY)
    
    # #save to file
    # saveBitMap.SaveBitmapFile(saveDC, f'img_bank/pic_{title}.bmp')
    
    # mfcDC.DeleteDC()
    # saveDC.DeleteDC()
    # #realease resources
    # win32gui.ReleaseDC(hwin, hwndDC)
    # win32gui.DeleteObject(saveBitMap.GetHandle())

In [5]:
ls = list_visable_windows()

NameError: name 'name_list' is not defined

In [6]:
ls = list_visable_windows()

NameError: name 'name_list' is not defined

In [7]:
def list_visable_windows():
    """
    Print all visable windows.
    """
    win_list = []
    def winEnumHandler(hwnd, ctx):
        if win32gui.IsWindowVisible(hwnd):
            win_list.append(Window(hwnd))
            # print(hex(hwnd), win32gui.GetWindowText(hwnd))
    win32gui.EnumWindows(winEnumHandler, None)
    #remove empty strings
    win_list = [x for x in win_list if x.name != '']
    return win_list    

def grab_screen():
    w = 1920 # set this
    h = 1080 # set this
    bmpfilenamename = "out.bmp" #set this

    hwnd= None #desktop
    wDC = win32gui.GetWindowDC(hwnd) # type: ignore
    dcObj=win32ui.CreateDCFromHandle(wDC)
    cDC=dcObj.CreateCompatibleDC()
    dataBitMap = win32ui.CreateBitmap()
    dataBitMap.CreateCompatibleBitmap(dcObj, w, h)
    cDC.SelectObject(dataBitMap)
    cDC.BitBlt((0,0),(w, h) , dcObj, (0,0), win32con.SRCCOPY)
    dataBitMap.SaveBitmapFile(cDC, bmpfilenamename)

    # Free Resources
    dcObj.DeleteDC()
    cDC.DeleteDC()
    win32gui.ReleaseDC(hwnd, wDC) # type: ignore
    win32gui.DeleteObject(dataBitMap.GetHandle())

def grab_window_bitmap(title = None):  # sourcery skip: raise-specific-error
    """
    Grab a screenshot of the specified window.

    Args:
        title (_type_, optional): _description_. Defaults to None.
    """
    
    w = 1920 # set this
    h = 1080 # set this
    # get the window handle
    hwnd = win32gui.FindWindow(None, title)
    if not hwnd:
        raise Exception(f"Window {title} not found.")
    '''
    cDC.SelectObject(dataBitMap)
    cDC.BitBlt((0, 0), (self.w, self.h), dcObj, (self.cropped_x, self.cropped_y), win32con.SRCCOPY)
    '''
    # get the window size
    react = win32gui.GetWindowRect(hwnd)
    # left, top, right, bot = win32gui.GetWindowRect(hwnd)
    # w = right - left
    # h = bot - top
    
    wDC = win32gui.GetWindowDC(hwnd) # type: ignore
    dcObj=win32ui.CreateDCFromHandle(wDC)
    cDC=dcObj.CreateCompatibleDC()
    dataBitMap = win32ui.CreateBitmap()
    dataBitMap.CreateCompatibleBitmap(dcObj, w, h)
    cDC.SelectObject(dataBitMap)
    cDC.BitBlt((0,0),(w, h) , dcObj, (0,0), win32con.SRCCOPY)
    signedIntsArray = dataBitMap.GetBitmapBits(True)
    # Free Resources
    dcObj.DeleteDC()
    cDC.DeleteDC()
    win32gui.ReleaseDC(hwnd, wDC) # type: ignore
    win32gui.DeleteObject(dataBitMap.GetHandle())
    img = np.fromstring(signedIntsArray, dtype='uint8') # type: ignore
    
    img.shape = (h,w,4)
    # cut off the alpha channel
    # img = img[:,:,:3]
    return img
    
def grab_save_window_screenshot(hwnd=None):
    
    """
    Grab a screenshot of the current window.
    """
    w = 1920 # set this
    h = 1080 # set this
    # get the window handle
    # hwnd = win32gui.FindWindow(None, title)
    # if not hwnd:
    #     raise Exception(f"Window {title} not found.")
    '''
    cDC.SelectObject(dataBitMap)
    cDC.BitBlt((0, 0), (self.w, self.h), dcObj, (self.cropped_x, self.cropped_y), win32con.SRCCOPY)
    '''
    # get the window size
    react = win32gui.GetWindowRect(hwnd) # type: ignore
    left, top, right, bot = win32gui.GetWindowRect(hwnd) # type: ignore
    w = right - left
    h = bot - top
    
    wDC = win32gui.GetWindowDC(hwnd) # type: ignore
    dcObj=win32ui.CreateDCFromHandle(wDC)
    cDC=dcObj.CreateCompatibleDC()
    dataBitMap = win32ui.CreateBitmap()
    dataBitMap.CreateCompatibleBitmap(dcObj, w, h)
    cDC.SelectObject(dataBitMap)
    cDC.BitBlt((left,top),(w, h) , dcObj, (0,0), win32con.SRCCOPY)
    dataBitMap.SaveBitmapFile(cDC, f'img_bank/pic_{win32gui.GetWindowText(hwnd)}_{hwnd}.bmp') # type: ignore
    # Free Resources
    dcObj.DeleteDC()
    cDC.DeleteDC()
    win32gui.ReleaseDC(hwnd, wDC) # type: ignore
    win32gui.DeleteObject(dataBitMap.GetHandle())
    # img = np.fromstring(signedIntsArray, dtype='uint8') # type: ignore
    # img.shape = (h,w,4)
    # cut off the alpha channel
    # img = img[:,:,:3]
    # return img
    # ------------------------------------
    # hwin = None
    # if title == None:
    #     hwin = win32gui.FindWindow(None, title)
    
    # left, top, right, bot = win32gui.GetWindowRect(hwin)
    # w = 1920
    # h = 1080
    
    # # save the screen
    # hwndDC = win32gui.GetWindowDC(hwin)
    # mfcDC  = win32ui.CreateDCFromHandle(hwndDC)
    # saveDC = mfcDC.CreateCompatibleDC()
    # saveBitMap = win32ui.CreateBitmap()
    # saveBitMap.CreateCompatibleBitmap(mfcDC, w, h)
    # saveDC.SelectObject(saveBitMap)
    # saveDC.BitBlt((0, 0), (w, h),  mfcDC,  (0, 0),  win32con.SRCCOPY)
    
    # #save to file
    # saveBitMap.SaveBitmapFile(saveDC, f'img_bank/pic_{title}.bmp')
    
    # mfcDC.DeleteDC()
    # saveDC.DeleteDC()
    # #realease resources
    # win32gui.ReleaseDC(hwin, hwndDC)
    # win32gui.DeleteObject(saveBitMap.GetHandle())

In [8]:
ls = list_visable_windows()

AttributeError: 'Window' object has no attribute 'name'

In [9]:
def list_visable_windows():
    """
    Print all visable windows.
    """
    win_list = []
    def winEnumHandler(hwnd, ctx):
        if win32gui.IsWindowVisible(hwnd):
            win_list.append(Window(hwnd))
            # print(hex(hwnd), win32gui.GetWindowText(hwnd))
    win32gui.EnumWindows(winEnumHandler, None)
    #remove empty strings
    win_list = [x for x in win_list if x.text != '']
    return win_list    

def grab_screen():
    w = 1920 # set this
    h = 1080 # set this
    bmpfilenamename = "out.bmp" #set this

    hwnd= None #desktop
    wDC = win32gui.GetWindowDC(hwnd) # type: ignore
    dcObj=win32ui.CreateDCFromHandle(wDC)
    cDC=dcObj.CreateCompatibleDC()
    dataBitMap = win32ui.CreateBitmap()
    dataBitMap.CreateCompatibleBitmap(dcObj, w, h)
    cDC.SelectObject(dataBitMap)
    cDC.BitBlt((0,0),(w, h) , dcObj, (0,0), win32con.SRCCOPY)
    dataBitMap.SaveBitmapFile(cDC, bmpfilenamename)

    # Free Resources
    dcObj.DeleteDC()
    cDC.DeleteDC()
    win32gui.ReleaseDC(hwnd, wDC) # type: ignore
    win32gui.DeleteObject(dataBitMap.GetHandle())

def grab_window_bitmap(title = None):  # sourcery skip: raise-specific-error
    """
    Grab a screenshot of the specified window.

    Args:
        title (_type_, optional): _description_. Defaults to None.
    """
    
    w = 1920 # set this
    h = 1080 # set this
    # get the window handle
    hwnd = win32gui.FindWindow(None, title)
    if not hwnd:
        raise Exception(f"Window {title} not found.")
    '''
    cDC.SelectObject(dataBitMap)
    cDC.BitBlt((0, 0), (self.w, self.h), dcObj, (self.cropped_x, self.cropped_y), win32con.SRCCOPY)
    '''
    # get the window size
    react = win32gui.GetWindowRect(hwnd)
    # left, top, right, bot = win32gui.GetWindowRect(hwnd)
    # w = right - left
    # h = bot - top
    
    wDC = win32gui.GetWindowDC(hwnd) # type: ignore
    dcObj=win32ui.CreateDCFromHandle(wDC)
    cDC=dcObj.CreateCompatibleDC()
    dataBitMap = win32ui.CreateBitmap()
    dataBitMap.CreateCompatibleBitmap(dcObj, w, h)
    cDC.SelectObject(dataBitMap)
    cDC.BitBlt((0,0),(w, h) , dcObj, (0,0), win32con.SRCCOPY)
    signedIntsArray = dataBitMap.GetBitmapBits(True)
    # Free Resources
    dcObj.DeleteDC()
    cDC.DeleteDC()
    win32gui.ReleaseDC(hwnd, wDC) # type: ignore
    win32gui.DeleteObject(dataBitMap.GetHandle())
    img = np.fromstring(signedIntsArray, dtype='uint8') # type: ignore
    
    img.shape = (h,w,4)
    # cut off the alpha channel
    # img = img[:,:,:3]
    return img
    
def grab_save_window_screenshot(hwnd=None):
    
    """
    Grab a screenshot of the current window.
    """
    w = 1920 # set this
    h = 1080 # set this
    # get the window handle
    # hwnd = win32gui.FindWindow(None, title)
    # if not hwnd:
    #     raise Exception(f"Window {title} not found.")
    '''
    cDC.SelectObject(dataBitMap)
    cDC.BitBlt((0, 0), (self.w, self.h), dcObj, (self.cropped_x, self.cropped_y), win32con.SRCCOPY)
    '''
    # get the window size
    react = win32gui.GetWindowRect(hwnd) # type: ignore
    left, top, right, bot = win32gui.GetWindowRect(hwnd) # type: ignore
    w = right - left
    h = bot - top
    
    wDC = win32gui.GetWindowDC(hwnd) # type: ignore
    dcObj=win32ui.CreateDCFromHandle(wDC)
    cDC=dcObj.CreateCompatibleDC()
    dataBitMap = win32ui.CreateBitmap()
    dataBitMap.CreateCompatibleBitmap(dcObj, w, h)
    cDC.SelectObject(dataBitMap)
    cDC.BitBlt((left,top),(w, h) , dcObj, (0,0), win32con.SRCCOPY)
    dataBitMap.SaveBitmapFile(cDC, f'img_bank/pic_{win32gui.GetWindowText(hwnd)}_{hwnd}.bmp') # type: ignore
    # Free Resources
    dcObj.DeleteDC()
    cDC.DeleteDC()
    win32gui.ReleaseDC(hwnd, wDC) # type: ignore
    win32gui.DeleteObject(dataBitMap.GetHandle())
    # img = np.fromstring(signedIntsArray, dtype='uint8') # type: ignore
    # img.shape = (h,w,4)
    # cut off the alpha channel
    # img = img[:,:,:3]
    # return img
    # ------------------------------------
    # hwin = None
    # if title == None:
    #     hwin = win32gui.FindWindow(None, title)
    
    # left, top, right, bot = win32gui.GetWindowRect(hwin)
    # w = 1920
    # h = 1080
    
    # # save the screen
    # hwndDC = win32gui.GetWindowDC(hwin)
    # mfcDC  = win32ui.CreateDCFromHandle(hwndDC)
    # saveDC = mfcDC.CreateCompatibleDC()
    # saveBitMap = win32ui.CreateBitmap()
    # saveBitMap.CreateCompatibleBitmap(mfcDC, w, h)
    # saveDC.SelectObject(saveBitMap)
    # saveDC.BitBlt((0, 0), (w, h),  mfcDC,  (0, 0),  win32con.SRCCOPY)
    
    # #save to file
    # saveBitMap.SaveBitmapFile(saveDC, f'img_bank/pic_{title}.bmp')
    
    # mfcDC.DeleteDC()
    # saveDC.DeleteDC()
    # #realease resources
    # win32gui.ReleaseDC(hwin, hwndDC)
    # win32gui.DeleteObject(saveBitMap.GetHandle())

In [10]:
ls = list_visable_windows()

AttributeError: 'Window' object has no attribute 'text'

In [11]:
import re
import win32gui, win32ui , win32con, win32api
import numpy as np
from dataclasses import dataclass, field ,fields

@dataclass
class Window:
    """
    Dataclass for window.
    """    
    text:   int
    title:  str = field(init=False)
    left:   int = field(init=False)
    top:    int = field(init=False)
    right:  int = field(init=False) 
    bot:    int = field(init=False)
    w:      int = field(init=False)
    h:      int = field(init=False)
    
    def __post_init__(self):
        self.title = win32gui.GetWindowText(self.hwnd)
        self.left, self.top, self.right, self.bot = win32gui.GetWindowRect(self.hwnd)
        w = self.right - self.left
        h = self.bot - self.top

In [12]:
def list_visable_windows():
    """
    Print all visable windows.
    """
    win_list = []
    def winEnumHandler(hwnd, ctx):
        if win32gui.IsWindowVisible(hwnd):
            win_list.append(Window(hwnd))
            # print(hex(hwnd), win32gui.GetWindowText(hwnd))
    win32gui.EnumWindows(winEnumHandler, None)
    #remove empty strings
    win_list = [x for x in win_list if x.text != '']
    return win_list    

def grab_screen():
    w = 1920 # set this
    h = 1080 # set this
    bmpfilenamename = "out.bmp" #set this

    hwnd= None #desktop
    wDC = win32gui.GetWindowDC(hwnd) # type: ignore
    dcObj=win32ui.CreateDCFromHandle(wDC)
    cDC=dcObj.CreateCompatibleDC()
    dataBitMap = win32ui.CreateBitmap()
    dataBitMap.CreateCompatibleBitmap(dcObj, w, h)
    cDC.SelectObject(dataBitMap)
    cDC.BitBlt((0,0),(w, h) , dcObj, (0,0), win32con.SRCCOPY)
    dataBitMap.SaveBitmapFile(cDC, bmpfilenamename)

    # Free Resources
    dcObj.DeleteDC()
    cDC.DeleteDC()
    win32gui.ReleaseDC(hwnd, wDC) # type: ignore
    win32gui.DeleteObject(dataBitMap.GetHandle())

def grab_window_bitmap(title = None):  # sourcery skip: raise-specific-error
    """
    Grab a screenshot of the specified window.

    Args:
        title (_type_, optional): _description_. Defaults to None.
    """
    
    w = 1920 # set this
    h = 1080 # set this
    # get the window handle
    hwnd = win32gui.FindWindow(None, title)
    if not hwnd:
        raise Exception(f"Window {title} not found.")
    '''
    cDC.SelectObject(dataBitMap)
    cDC.BitBlt((0, 0), (self.w, self.h), dcObj, (self.cropped_x, self.cropped_y), win32con.SRCCOPY)
    '''
    # get the window size
    react = win32gui.GetWindowRect(hwnd)
    # left, top, right, bot = win32gui.GetWindowRect(hwnd)
    # w = right - left
    # h = bot - top
    
    wDC = win32gui.GetWindowDC(hwnd) # type: ignore
    dcObj=win32ui.CreateDCFromHandle(wDC)
    cDC=dcObj.CreateCompatibleDC()
    dataBitMap = win32ui.CreateBitmap()
    dataBitMap.CreateCompatibleBitmap(dcObj, w, h)
    cDC.SelectObject(dataBitMap)
    cDC.BitBlt((0,0),(w, h) , dcObj, (0,0), win32con.SRCCOPY)
    signedIntsArray = dataBitMap.GetBitmapBits(True)
    # Free Resources
    dcObj.DeleteDC()
    cDC.DeleteDC()
    win32gui.ReleaseDC(hwnd, wDC) # type: ignore
    win32gui.DeleteObject(dataBitMap.GetHandle())
    img = np.fromstring(signedIntsArray, dtype='uint8') # type: ignore
    
    img.shape = (h,w,4)
    # cut off the alpha channel
    # img = img[:,:,:3]
    return img
    
def grab_save_window_screenshot(hwnd=None):
    
    """
    Grab a screenshot of the current window.
    """
    w = 1920 # set this
    h = 1080 # set this
    # get the window handle
    # hwnd = win32gui.FindWindow(None, title)
    # if not hwnd:
    #     raise Exception(f"Window {title} not found.")
    '''
    cDC.SelectObject(dataBitMap)
    cDC.BitBlt((0, 0), (self.w, self.h), dcObj, (self.cropped_x, self.cropped_y), win32con.SRCCOPY)
    '''
    # get the window size
    react = win32gui.GetWindowRect(hwnd) # type: ignore
    left, top, right, bot = win32gui.GetWindowRect(hwnd) # type: ignore
    w = right - left
    h = bot - top
    
    wDC = win32gui.GetWindowDC(hwnd) # type: ignore
    dcObj=win32ui.CreateDCFromHandle(wDC)
    cDC=dcObj.CreateCompatibleDC()
    dataBitMap = win32ui.CreateBitmap()
    dataBitMap.CreateCompatibleBitmap(dcObj, w, h)
    cDC.SelectObject(dataBitMap)
    cDC.BitBlt((left,top),(w, h) , dcObj, (0,0), win32con.SRCCOPY)
    dataBitMap.SaveBitmapFile(cDC, f'img_bank/pic_{win32gui.GetWindowText(hwnd)}_{hwnd}.bmp') # type: ignore
    # Free Resources
    dcObj.DeleteDC()
    cDC.DeleteDC()
    win32gui.ReleaseDC(hwnd, wDC) # type: ignore
    win32gui.DeleteObject(dataBitMap.GetHandle())
    # img = np.fromstring(signedIntsArray, dtype='uint8') # type: ignore
    # img.shape = (h,w,4)
    # cut off the alpha channel
    # img = img[:,:,:3]
    # return img
    # ------------------------------------
    # hwin = None
    # if title == None:
    #     hwin = win32gui.FindWindow(None, title)
    
    # left, top, right, bot = win32gui.GetWindowRect(hwin)
    # w = 1920
    # h = 1080
    
    # # save the screen
    # hwndDC = win32gui.GetWindowDC(hwin)
    # mfcDC  = win32ui.CreateDCFromHandle(hwndDC)
    # saveDC = mfcDC.CreateCompatibleDC()
    # saveBitMap = win32ui.CreateBitmap()
    # saveBitMap.CreateCompatibleBitmap(mfcDC, w, h)
    # saveDC.SelectObject(saveBitMap)
    # saveDC.BitBlt((0, 0), (w, h),  mfcDC,  (0, 0),  win32con.SRCCOPY)
    
    # #save to file
    # saveBitMap.SaveBitmapFile(saveDC, f'img_bank/pic_{title}.bmp')
    
    # mfcDC.DeleteDC()
    # saveDC.DeleteDC()
    # #realease resources
    # win32gui.ReleaseDC(hwin, hwndDC)
    # win32gui.DeleteObject(saveBitMap.GetHandle())

In [13]:

ls = list_visable_windows()

AttributeError: 'Window' object has no attribute 'hwnd'

In [14]:
import re
import win32gui, win32ui , win32con, win32api
import numpy as np
from dataclasses import dataclass, field ,fields

@dataclass
class Window:
    """
    Dataclass for window.
    """    
    hwnd:   int
    text:  str = field(init=False)
    left:   int = field(init=False)
    top:    int = field(init=False)
    right:  int = field(init=False) 
    bot:    int = field(init=False)
    w:      int = field(init=False)
    h:      int = field(init=False)
    
    def __post_init__(self):
        self.title = win32gui.GetWindowText(self.hwnd)
        self.left, self.top, self.right, self.bot = win32gui.GetWindowRect(self.hwnd)
        w = self.right - self.left
        h = self.bot - self.top

In [15]:
def list_visable_windows():
    """
    Print all visable windows.
    """
    win_list = []
    def winEnumHandler(hwnd, ctx):
        if win32gui.IsWindowVisible(hwnd):
            win_list.append(Window(hwnd))
            # print(hex(hwnd), win32gui.GetWindowText(hwnd))
    win32gui.EnumWindows(winEnumHandler, None)
    #remove empty strings
    win_list = [x for x in win_list if x.text != '']
    return win_list    

def grab_screen():
    w = 1920 # set this
    h = 1080 # set this
    bmpfilenamename = "out.bmp" #set this

    hwnd= None #desktop
    wDC = win32gui.GetWindowDC(hwnd) # type: ignore
    dcObj=win32ui.CreateDCFromHandle(wDC)
    cDC=dcObj.CreateCompatibleDC()
    dataBitMap = win32ui.CreateBitmap()
    dataBitMap.CreateCompatibleBitmap(dcObj, w, h)
    cDC.SelectObject(dataBitMap)
    cDC.BitBlt((0,0),(w, h) , dcObj, (0,0), win32con.SRCCOPY)
    dataBitMap.SaveBitmapFile(cDC, bmpfilenamename)

    # Free Resources
    dcObj.DeleteDC()
    cDC.DeleteDC()
    win32gui.ReleaseDC(hwnd, wDC) # type: ignore
    win32gui.DeleteObject(dataBitMap.GetHandle())

def grab_window_bitmap(title = None):  # sourcery skip: raise-specific-error
    """
    Grab a screenshot of the specified window.

    Args:
        title (_type_, optional): _description_. Defaults to None.
    """
    
    w = 1920 # set this
    h = 1080 # set this
    # get the window handle
    hwnd = win32gui.FindWindow(None, title)
    if not hwnd:
        raise Exception(f"Window {title} not found.")
    '''
    cDC.SelectObject(dataBitMap)
    cDC.BitBlt((0, 0), (self.w, self.h), dcObj, (self.cropped_x, self.cropped_y), win32con.SRCCOPY)
    '''
    # get the window size
    react = win32gui.GetWindowRect(hwnd)
    # left, top, right, bot = win32gui.GetWindowRect(hwnd)
    # w = right - left
    # h = bot - top
    
    wDC = win32gui.GetWindowDC(hwnd) # type: ignore
    dcObj=win32ui.CreateDCFromHandle(wDC)
    cDC=dcObj.CreateCompatibleDC()
    dataBitMap = win32ui.CreateBitmap()
    dataBitMap.CreateCompatibleBitmap(dcObj, w, h)
    cDC.SelectObject(dataBitMap)
    cDC.BitBlt((0,0),(w, h) , dcObj, (0,0), win32con.SRCCOPY)
    signedIntsArray = dataBitMap.GetBitmapBits(True)
    # Free Resources
    dcObj.DeleteDC()
    cDC.DeleteDC()
    win32gui.ReleaseDC(hwnd, wDC) # type: ignore
    win32gui.DeleteObject(dataBitMap.GetHandle())
    img = np.fromstring(signedIntsArray, dtype='uint8') # type: ignore
    
    img.shape = (h,w,4)
    # cut off the alpha channel
    # img = img[:,:,:3]
    return img
    
def grab_save_window_screenshot(hwnd=None):
    
    """
    Grab a screenshot of the current window.
    """
    w = 1920 # set this
    h = 1080 # set this
    # get the window handle
    # hwnd = win32gui.FindWindow(None, title)
    # if not hwnd:
    #     raise Exception(f"Window {title} not found.")
    '''
    cDC.SelectObject(dataBitMap)
    cDC.BitBlt((0, 0), (self.w, self.h), dcObj, (self.cropped_x, self.cropped_y), win32con.SRCCOPY)
    '''
    # get the window size
    react = win32gui.GetWindowRect(hwnd) # type: ignore
    left, top, right, bot = win32gui.GetWindowRect(hwnd) # type: ignore
    w = right - left
    h = bot - top
    
    wDC = win32gui.GetWindowDC(hwnd) # type: ignore
    dcObj=win32ui.CreateDCFromHandle(wDC)
    cDC=dcObj.CreateCompatibleDC()
    dataBitMap = win32ui.CreateBitmap()
    dataBitMap.CreateCompatibleBitmap(dcObj, w, h)
    cDC.SelectObject(dataBitMap)
    cDC.BitBlt((left,top),(w, h) , dcObj, (0,0), win32con.SRCCOPY)
    dataBitMap.SaveBitmapFile(cDC, f'img_bank/pic_{win32gui.GetWindowText(hwnd)}_{hwnd}.bmp') # type: ignore
    # Free Resources
    dcObj.DeleteDC()
    cDC.DeleteDC()
    win32gui.ReleaseDC(hwnd, wDC) # type: ignore
    win32gui.DeleteObject(dataBitMap.GetHandle())
    # img = np.fromstring(signedIntsArray, dtype='uint8') # type: ignore
    # img.shape = (h,w,4)
    # cut off the alpha channel
    # img = img[:,:,:3]
    # return img
    # ------------------------------------
    # hwin = None
    # if title == None:
    #     hwin = win32gui.FindWindow(None, title)
    
    # left, top, right, bot = win32gui.GetWindowRect(hwin)
    # w = 1920
    # h = 1080
    
    # # save the screen
    # hwndDC = win32gui.GetWindowDC(hwin)
    # mfcDC  = win32ui.CreateDCFromHandle(hwndDC)
    # saveDC = mfcDC.CreateCompatibleDC()
    # saveBitMap = win32ui.CreateBitmap()
    # saveBitMap.CreateCompatibleBitmap(mfcDC, w, h)
    # saveDC.SelectObject(saveBitMap)
    # saveDC.BitBlt((0, 0), (w, h),  mfcDC,  (0, 0),  win32con.SRCCOPY)
    
    # #save to file
    # saveBitMap.SaveBitmapFile(saveDC, f'img_bank/pic_{title}.bmp')
    
    # mfcDC.DeleteDC()
    # saveDC.DeleteDC()
    # #realease resources
    # win32gui.ReleaseDC(hwin, hwndDC)
    # win32gui.DeleteObject(saveBitMap.GetHandle())

In [16]:

ls = list_visable_windows()

AttributeError: 'Window' object has no attribute 'text'

In [17]:
import re
import win32gui, win32ui , win32con, win32api
import numpy as np
from dataclasses import dataclass, field ,fields

@dataclass
class Window:
    """
    Dataclass for window.
    """    
    hwnd:   int
    text:  str = field(init=False)
    left:   int = field(init=False)
    top:    int = field(init=False)
    right:  int = field(init=False) 
    bot:    int = field(init=False)
    w:      int = field(init=False)
    h:      int = field(init=False)
    
    def __post_init__(self):
        self.title = win32gui.GetWindowText(self.hwnd)
        self.left, self.top, self.right, self.bot = win32gui.GetWindowRect(self.hwnd)
        w = self.right - self.left
        h = self.bot - self.top

In [18]:
import re
import win32gui, win32ui , win32con, win32api
import numpy as np
from dataclasses import dataclass, field ,fields

@dataclass
class Window:
    """
    Dataclass for window.
    """    
    hwnd:   int
    text:  str = field(init=False)
    left:   int = field(init=False)
    top:    int = field(init=False)
    right:  int = field(init=False) 
    bot:    int = field(init=False)
    w:      int = field(init=False)
    h:      int = field(init=False)
    
    def __post_init__(self):
        self.text = win32gui.GetWindowText(self.hwnd)
        self.left, self.top, self.right, self.bot = win32gui.GetWindowRect(self.hwnd)
        w = self.right - self.left
        h = self.bot - self.top

In [19]:
def list_visable_windows():
    """
    Print all visable windows.
    """
    win_list = []
    def winEnumHandler(hwnd, ctx):
        if win32gui.IsWindowVisible(hwnd):
            win_list.append(Window(hwnd))
            # print(hex(hwnd), win32gui.GetWindowText(hwnd))
    win32gui.EnumWindows(winEnumHandler, None)
    #remove empty strings
    win_list = [x for x in win_list if x.text != '']
    return win_list    

def grab_screen():
    w = 1920 # set this
    h = 1080 # set this
    bmpfilenamename = "out.bmp" #set this

    hwnd= None #desktop
    wDC = win32gui.GetWindowDC(hwnd) # type: ignore
    dcObj=win32ui.CreateDCFromHandle(wDC)
    cDC=dcObj.CreateCompatibleDC()
    dataBitMap = win32ui.CreateBitmap()
    dataBitMap.CreateCompatibleBitmap(dcObj, w, h)
    cDC.SelectObject(dataBitMap)
    cDC.BitBlt((0,0),(w, h) , dcObj, (0,0), win32con.SRCCOPY)
    dataBitMap.SaveBitmapFile(cDC, bmpfilenamename)

    # Free Resources
    dcObj.DeleteDC()
    cDC.DeleteDC()
    win32gui.ReleaseDC(hwnd, wDC) # type: ignore
    win32gui.DeleteObject(dataBitMap.GetHandle())

def grab_window_bitmap(title = None):  # sourcery skip: raise-specific-error
    """
    Grab a screenshot of the specified window.

    Args:
        title (_type_, optional): _description_. Defaults to None.
    """
    
    w = 1920 # set this
    h = 1080 # set this
    # get the window handle
    hwnd = win32gui.FindWindow(None, title)
    if not hwnd:
        raise Exception(f"Window {title} not found.")
    '''
    cDC.SelectObject(dataBitMap)
    cDC.BitBlt((0, 0), (self.w, self.h), dcObj, (self.cropped_x, self.cropped_y), win32con.SRCCOPY)
    '''
    # get the window size
    react = win32gui.GetWindowRect(hwnd)
    # left, top, right, bot = win32gui.GetWindowRect(hwnd)
    # w = right - left
    # h = bot - top
    
    wDC = win32gui.GetWindowDC(hwnd) # type: ignore
    dcObj=win32ui.CreateDCFromHandle(wDC)
    cDC=dcObj.CreateCompatibleDC()
    dataBitMap = win32ui.CreateBitmap()
    dataBitMap.CreateCompatibleBitmap(dcObj, w, h)
    cDC.SelectObject(dataBitMap)
    cDC.BitBlt((0,0),(w, h) , dcObj, (0,0), win32con.SRCCOPY)
    signedIntsArray = dataBitMap.GetBitmapBits(True)
    # Free Resources
    dcObj.DeleteDC()
    cDC.DeleteDC()
    win32gui.ReleaseDC(hwnd, wDC) # type: ignore
    win32gui.DeleteObject(dataBitMap.GetHandle())
    img = np.fromstring(signedIntsArray, dtype='uint8') # type: ignore
    
    img.shape = (h,w,4)
    # cut off the alpha channel
    # img = img[:,:,:3]
    return img
    
def grab_save_window_screenshot(hwnd=None):
    
    """
    Grab a screenshot of the current window.
    """
    w = 1920 # set this
    h = 1080 # set this
    # get the window handle
    # hwnd = win32gui.FindWindow(None, title)
    # if not hwnd:
    #     raise Exception(f"Window {title} not found.")
    '''
    cDC.SelectObject(dataBitMap)
    cDC.BitBlt((0, 0), (self.w, self.h), dcObj, (self.cropped_x, self.cropped_y), win32con.SRCCOPY)
    '''
    # get the window size
    react = win32gui.GetWindowRect(hwnd) # type: ignore
    left, top, right, bot = win32gui.GetWindowRect(hwnd) # type: ignore
    w = right - left
    h = bot - top
    
    wDC = win32gui.GetWindowDC(hwnd) # type: ignore
    dcObj=win32ui.CreateDCFromHandle(wDC)
    cDC=dcObj.CreateCompatibleDC()
    dataBitMap = win32ui.CreateBitmap()
    dataBitMap.CreateCompatibleBitmap(dcObj, w, h)
    cDC.SelectObject(dataBitMap)
    cDC.BitBlt((left,top),(w, h) , dcObj, (0,0), win32con.SRCCOPY)
    dataBitMap.SaveBitmapFile(cDC, f'img_bank/pic_{win32gui.GetWindowText(hwnd)}_{hwnd}.bmp') # type: ignore
    # Free Resources
    dcObj.DeleteDC()
    cDC.DeleteDC()
    win32gui.ReleaseDC(hwnd, wDC) # type: ignore
    win32gui.DeleteObject(dataBitMap.GetHandle())
    # img = np.fromstring(signedIntsArray, dtype='uint8') # type: ignore
    # img.shape = (h,w,4)
    # cut off the alpha channel
    # img = img[:,:,:3]
    # return img
    # ------------------------------------
    # hwin = None
    # if title == None:
    #     hwin = win32gui.FindWindow(None, title)
    
    # left, top, right, bot = win32gui.GetWindowRect(hwin)
    # w = 1920
    # h = 1080
    
    # # save the screen
    # hwndDC = win32gui.GetWindowDC(hwin)
    # mfcDC  = win32ui.CreateDCFromHandle(hwndDC)
    # saveDC = mfcDC.CreateCompatibleDC()
    # saveBitMap = win32ui.CreateBitmap()
    # saveBitMap.CreateCompatibleBitmap(mfcDC, w, h)
    # saveDC.SelectObject(saveBitMap)
    # saveDC.BitBlt((0, 0), (w, h),  mfcDC,  (0, 0),  win32con.SRCCOPY)
    
    # #save to file
    # saveBitMap.SaveBitmapFile(saveDC, f'img_bank/pic_{title}.bmp')
    
    # mfcDC.DeleteDC()
    # saveDC.DeleteDC()
    # #realease resources
    # win32gui.ReleaseDC(hwin, hwndDC)
    # win32gui.DeleteObject(saveBitMap.GetHandle())

In [20]:

ls = list_visable_windows()

In [21]:
print ls

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(...)? (<ipython-input-21-8ed24599bc6f>, line 1)

In [22]:
ls

AttributeError: 'Window' object has no attribute 'w'

In [23]:
import re
import win32gui, win32ui , win32con, win32api
import numpy as np
from dataclasses import dataclass, field ,fields

@dataclass
class Window:
    """
    Dataclass for window.
    """    
    hwnd:   int
    text:  str = field(init=False)
    left:   int = field(init=False)
    top:    int = field(init=False)
    right:  int = field(init=False) 
    bot:    int = field(init=False)
    width:      int = field(init=False)
    height:      int = field(init=False)
    
    def __post_init__(self):
        self.text = win32gui.GetWindowText(self.hwnd)
        self.left, self.top, self.right, self.bot = win32gui.GetWindowRect(self.hwnd)
        self.width = self.right - self.left
        self.height = self.bot - self.top

In [24]:
def list_visable_windows():
    """
    Print all visable windows.
    """
    win_list = []
    def winEnumHandler(hwnd, ctx):
        if win32gui.IsWindowVisible(hwnd):
            win_list.append(Window(hwnd))
            # print(hex(hwnd), win32gui.GetWindowText(hwnd))
    win32gui.EnumWindows(winEnumHandler, None)
    #remove empty strings
    win_list = [x for x in win_list if x.text != '']
    return win_list    

def grab_screen():
    w = 1920 # set this
    h = 1080 # set this
    bmpfilenamename = "out.bmp" #set this

    hwnd= None #desktop
    wDC = win32gui.GetWindowDC(hwnd) # type: ignore
    dcObj=win32ui.CreateDCFromHandle(wDC)
    cDC=dcObj.CreateCompatibleDC()
    dataBitMap = win32ui.CreateBitmap()
    dataBitMap.CreateCompatibleBitmap(dcObj, w, h)
    cDC.SelectObject(dataBitMap)
    cDC.BitBlt((0,0),(w, h) , dcObj, (0,0), win32con.SRCCOPY)
    dataBitMap.SaveBitmapFile(cDC, bmpfilenamename)

    # Free Resources
    dcObj.DeleteDC()
    cDC.DeleteDC()
    win32gui.ReleaseDC(hwnd, wDC) # type: ignore
    win32gui.DeleteObject(dataBitMap.GetHandle())

def grab_window_bitmap(title = None):  # sourcery skip: raise-specific-error
    """
    Grab a screenshot of the specified window.

    Args:
        title (_type_, optional): _description_. Defaults to None.
    """
    
    w = 1920 # set this
    h = 1080 # set this
    # get the window handle
    hwnd = win32gui.FindWindow(None, title)
    if not hwnd:
        raise Exception(f"Window {title} not found.")
    '''
    cDC.SelectObject(dataBitMap)
    cDC.BitBlt((0, 0), (self.w, self.h), dcObj, (self.cropped_x, self.cropped_y), win32con.SRCCOPY)
    '''
    # get the window size
    react = win32gui.GetWindowRect(hwnd)
    # left, top, right, bot = win32gui.GetWindowRect(hwnd)
    # w = right - left
    # h = bot - top
    
    wDC = win32gui.GetWindowDC(hwnd) # type: ignore
    dcObj=win32ui.CreateDCFromHandle(wDC)
    cDC=dcObj.CreateCompatibleDC()
    dataBitMap = win32ui.CreateBitmap()
    dataBitMap.CreateCompatibleBitmap(dcObj, w, h)
    cDC.SelectObject(dataBitMap)
    cDC.BitBlt((0,0),(w, h) , dcObj, (0,0), win32con.SRCCOPY)
    signedIntsArray = dataBitMap.GetBitmapBits(True)
    # Free Resources
    dcObj.DeleteDC()
    cDC.DeleteDC()
    win32gui.ReleaseDC(hwnd, wDC) # type: ignore
    win32gui.DeleteObject(dataBitMap.GetHandle())
    img = np.fromstring(signedIntsArray, dtype='uint8') # type: ignore
    
    img.shape = (h,w,4)
    # cut off the alpha channel
    # img = img[:,:,:3]
    return img
    
def grab_save_window_screenshot(hwnd=None):
    
    """
    Grab a screenshot of the current window.
    """
    w = 1920 # set this
    h = 1080 # set this
    # get the window handle
    # hwnd = win32gui.FindWindow(None, title)
    # if not hwnd:
    #     raise Exception(f"Window {title} not found.")
    '''
    cDC.SelectObject(dataBitMap)
    cDC.BitBlt((0, 0), (self.w, self.h), dcObj, (self.cropped_x, self.cropped_y), win32con.SRCCOPY)
    '''
    # get the window size
    react = win32gui.GetWindowRect(hwnd) # type: ignore
    left, top, right, bot = win32gui.GetWindowRect(hwnd) # type: ignore
    w = right - left
    h = bot - top
    
    wDC = win32gui.GetWindowDC(hwnd) # type: ignore
    dcObj=win32ui.CreateDCFromHandle(wDC)
    cDC=dcObj.CreateCompatibleDC()
    dataBitMap = win32ui.CreateBitmap()
    dataBitMap.CreateCompatibleBitmap(dcObj, w, h)
    cDC.SelectObject(dataBitMap)
    cDC.BitBlt((left,top),(w, h) , dcObj, (0,0), win32con.SRCCOPY)
    dataBitMap.SaveBitmapFile(cDC, f'img_bank/pic_{win32gui.GetWindowText(hwnd)}_{hwnd}.bmp') # type: ignore
    # Free Resources
    dcObj.DeleteDC()
    cDC.DeleteDC()
    win32gui.ReleaseDC(hwnd, wDC) # type: ignore
    win32gui.DeleteObject(dataBitMap.GetHandle())
    # img = np.fromstring(signedIntsArray, dtype='uint8') # type: ignore
    # img.shape = (h,w,4)
    # cut off the alpha channel
    # img = img[:,:,:3]
    # return img
    # ------------------------------------
    # hwin = None
    # if title == None:
    #     hwin = win32gui.FindWindow(None, title)
    
    # left, top, right, bot = win32gui.GetWindowRect(hwin)
    # w = 1920
    # h = 1080
    
    # # save the screen
    # hwndDC = win32gui.GetWindowDC(hwin)
    # mfcDC  = win32ui.CreateDCFromHandle(hwndDC)
    # saveDC = mfcDC.CreateCompatibleDC()
    # saveBitMap = win32ui.CreateBitmap()
    # saveBitMap.CreateCompatibleBitmap(mfcDC, w, h)
    # saveDC.SelectObject(saveBitMap)
    # saveDC.BitBlt((0, 0), (w, h),  mfcDC,  (0, 0),  win32con.SRCCOPY)
    
    # #save to file
    # saveBitMap.SaveBitmapFile(saveDC, f'img_bank/pic_{title}.bmp')
    
    # mfcDC.DeleteDC()
    # saveDC.DeleteDC()
    # #realease resources
    # win32gui.ReleaseDC(hwin, hwndDC)
    # win32gui.DeleteObject(saveBitMap.GetHandle())

In [28]:

ls = list_visable_windows()

In [29]:
ls

[Window(hwnd=3016040, text='Interactive - window_utils.py - ShinyHunterPuppetMaster - Visual Studio Code', left=0, top=0, right=1920, bot=1080, width=1920, height=1080),
 Window(hwnd=3147284, text='python - Dataclasses and property decorator - Stack Overflow - Google Chrome', left=-8, top=-8, right=1928, bot=1088, width=1936, height=1096),
 Window(hwnd=332934, text='Home', left=0, top=0, right=1649, bot=723, width=1649, height=723),
 Window(hwnd=460422, text='@BonBon - Discord', left=-8, top=-8, right=1928, bot=1088, width=1936, height=1096),
 Window(hwnd=1053762, text='LennyTheIntern/ShinyHunterPuppetMaster - Google Chrome', left=-8, top=-8, right=1928, bot=1088, width=1936, height=1096),
 Window(hwnd=68052, text='k - shiny Hunter - personal_notes_2_0 - Obsidian v1.3.7', left=-8, top=-8, right=1928, bot=1088, width=1936, height=1096),
 Window(hwnd=2426672, text='RoyalSapphireVA - Twitch - Google Chrome', left=-8, top=-8, right=1928, bot=1088, width=1936, height=1096),
 Window(hwnd=367

In [27]:
import re
import win32gui, win32ui , win32con, win32api
import numpy as np
from dataclasses import dataclass, field ,fields

@dataclass
class Window:
    """
    Dataclass for window.
    """    
    hwnd:   int
    text:   str = field(init=False)
    left:   int = field(init=False)
    top:    int = field(init=False)
    right:  int = field(init=False) 
    bot:    int = field(init=False)
    width:  int = field(init=False)
    height: int = field(init=False)
    
    def __post_init__(self):
        self.text = win32gui.GetWindowText(self.hwnd)
        self.left, self.top, self.right, self.bot = win32gui.GetWindowRect(self.hwnd)
        self.width = self.right - self.left
        self.height = self.bot - self.top
    
    @property
    def np_bitmap(self):
        wDC = win32gui.GetWindowDC(self.hwnd) # type: ignore
        dcObj=win32ui.CreateDCFromHandle(wDC)
        cDC=dcObj.CreateCompatibleDC()
        dataBitMap = win32ui.CreateBitmap()
        dataBitMap.CreateCompatibleBitmap(dcObj, self.width, self.height)
        cDC.SelectObject(dataBitMap)
        cDC.BitBlt((0,0),(self.width, self.height) , dcObj, (0,0), win32con.SRCCOPY)
        signedIntsArray = dataBitMap.GetBitmapBits(True)
        # Free Resources
        dcObj.DeleteDC()
        cDC.DeleteDC()
        win32gui.ReleaseDC(hwnd, wDC) # type: ignore
        win32gui.DeleteObject(dataBitMap.GetHandle())
        img = np.fromstring(signedIntsArray, dtype='uint8') # type: ignore
        
        img.shape = (self.height,self.width,4)
        # cut off the alpha channel
        # img = img[:,:,:3]
        return img

In [30]:
import re
import win32gui, win32ui , win32con, win32api
import numpy as np
from dataclasses import dataclass, field ,fields

@dataclass
class Window:
    """
    Dataclass for window.
    """    
    hwnd:   int
    text:   str = field(init=False)
    left:   int = field(init=False)
    top:    int = field(init=False)
    right:  int = field(init=False) 
    bot:    int = field(init=False)
    width:  int = field(init=False)
    height: int = field(init=False)
    
    def __post_init__(self):
        self.text = win32gui.GetWindowText(self.hwnd)
        self.left, self.top, self.right, self.bot = win32gui.GetWindowRect(self.hwnd)
        self.width = self.right - self.left
        self.height = self.bot - self.top
    
    @property
    def np_bitmap(self):
        wDC = win32gui.GetWindowDC(self.hwnd) # type: ignore
        dcObj=win32ui.CreateDCFromHandle(wDC)
        cDC=dcObj.CreateCompatibleDC()
        dataBitMap = win32ui.CreateBitmap()
        dataBitMap.CreateCompatibleBitmap(dcObj, self.width, self.height)
        cDC.SelectObject(dataBitMap)
        cDC.BitBlt((0,0),(self.width, self.height) , dcObj, (0,0), win32con.SRCCOPY)
        signedIntsArray = dataBitMap.GetBitmapBits(True)
        # Free Resources
        dcObj.DeleteDC()
        cDC.DeleteDC()
        win32gui.ReleaseDC(hwnd, wDC) # type: ignore
        win32gui.DeleteObject(dataBitMap.GetHandle())
        img = np.fromstring(signedIntsArray, dtype='uint8') # type: ignore
        
        img.shape = (self.height,self.width,4)
        # cut off the alpha channel
        # img = img[:,:,:3]
        return img

In [31]:

ls = list_visable_windows()

In [32]:
ls

[Window(hwnd=3016040, text='Interactive - window_utils.py - ShinyHunterPuppetMaster - Visual Studio Code', left=0, top=0, right=1920, bot=1080, width=1920, height=1080),
 Window(hwnd=3147284, text='python - Dataclasses and property decorator - Stack Overflow - Google Chrome', left=-8, top=-8, right=1928, bot=1088, width=1936, height=1096),
 Window(hwnd=332934, text='Home', left=0, top=0, right=1649, bot=723, width=1649, height=723),
 Window(hwnd=460422, text='@BonBon - Discord', left=-8, top=-8, right=1928, bot=1088, width=1936, height=1096),
 Window(hwnd=1053762, text='LennyTheIntern/ShinyHunterPuppetMaster - Google Chrome', left=-8, top=-8, right=1928, bot=1088, width=1936, height=1096),
 Window(hwnd=68052, text='k - shiny Hunter - personal_notes_2_0 - Obsidian v1.3.7', left=-8, top=-8, right=1928, bot=1088, width=1936, height=1096),
 Window(hwnd=2426672, text='RoyalSapphireVA - Twitch - Google Chrome', left=-8, top=-8, right=1928, bot=1088, width=1936, height=1096),
 Window(hwnd=367

In [33]:
w = ls[0]

In [34]:
w:Window = ls[0]